In [1]:
# collect all questions per paragraph id
from collections import defaultdict

paras = []
path = '../data/tasks15/test_paragraphs.tsv'
qp_ids = defaultdict(list)
p_ids = []

with open(path, encoding='utf-8') as f:
    for i, line in enumerate(f):
        q_id, para_id, para_text = line.rstrip().split('\t')
        if q_id not in qp_ids[para_id]:
            qp_ids[para_id].append(q_id)
        if para_text not in paras:
            paras.append(para_text)
            p_ids.append(para_id)
            
# print(len(paras))
print(len(p_ids), 'passages')
print(len(qp_ids), 'pages')

6225 passages
685 pages


In [3]:
# load all generated questions per passage
import numpy as np

qs_per_page = defaultdict(list)
with open('../results/tasks15/t5_tasks15_test_paragraphs_13.tsv', 'r') as fin:
    for i, line in enumerate(fin):
        cells = line.rstrip('\n').split('\t')
        # load generated questions
        p_id = p_ids[i]
        qs = cells[2:]
        qs_per_page[p_id].extend(qs)
print(len(qs_per_page), 'pages with generated questions')

601 pages with generated questions


In [3]:
# load gt questions
# collect all GS questions: independent
path_qs = '../data/tasks15/test_questions.tsv'

gs_qs = {}
with open(path_qs, 'r') as f:
#     next(f)
    for line in f:
        q_id, q = line.split('\t')
#         print(q_id)
        gs_qs[q_id] = (q.strip('\n'))
print(len(gs_qs), 'questions')
# print(gs_qs['1_11'])

547 questions


In [5]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [15]:
# evaluate generated questions
import numpy as np

max_scores_1f, max_scores_1r = [], []
max_scores_Lf, max_scores_Lr = [], []


# iterate over pages
print(len(qs_per_page), 'pages with generated questions')
n_generated = []
for page_id, qs in qs_per_page.items():
        # iterate over gt questions for each page
        q_ids = qp_ids[page_id]
#         print(page_id)
#         print(len(q_ids), 'gold truth questions for one page')
#         print(q_ids)
#         print(len(qs), 'generated questions for this page')
        n_generated.append(len(qs))
        for q_id in q_ids:
            gt_q = gs_qs[q_id]
            all_q_gens_scores_1f,  all_q_gens_scores_1r = [], []
            all_q_gens_scores_Lf,  all_q_gens_scores_Lr = [], []
            # iterate over generated questions for each page
            for q in qs:
                scores = scorer.score(gt_q, q)
                all_q_gens_scores_1f.append(scores['rouge1'].fmeasure)
                all_q_gens_scores_1r.append(scores['rouge1'].recall)
                all_q_gens_scores_Lf.append(scores['rougeL'].fmeasure)
                all_q_gens_scores_Lr.append(scores['rougeL'].recall)

            max_scores_1f.append(max(all_q_gens_scores_1f))
            max_scores_1r.append(max(all_q_gens_scores_1r))
            max_scores_Lf.append(max(all_q_gens_scores_Lf))
            max_scores_Lr.append(max(all_q_gens_scores_Lr))

# print(max_scores)
print(np.mean(max_scores_1f), 'R1-F')
print(np.mean(max_scores_1r), 'R1-R')
print(np.mean(max_scores_Lf), 'RL-F')
print(np.mean(max_scores_Lr), 'RL-R')

print(np.mean(n_generated))

601 pages with generated questions
0.36992105358364874
0.41466532354531843
0.3522205852388888
0.39196121113967203
134.65058236272878
